In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dash_table, dcc, html, Input, Output
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash
import pycountry_convert as pc
import base64


# Load the CSV data
data_url = 'https://query.data.world/s/ubyf7tmt7vkvimvz3u5kehrkx2rpwf?dws=00000'
df1 = pd.read_csv(data_url, encoding='latin-1')
df2 = pd.read_csv('./continents.csv')

df = pd.merge(df1, df2[["name", "region"]], left_on='Country', right_on='name')
#df = df3.drop('name', axis=1, inplace=True)

df3 = pd.read_csv(data_url, encoding='latin-1', usecols = ['ID', 'Country', 'Scheduled Delivery Date', 'Weight (Kilograms)'])
cols = df3.columns
cols = cols.map(lambda x: x.replace(' ', '_') if isinstance(x, (str)) else x)
df3.columns = cols
df3['Scheduled_Delivery_Date'] = pd.to_datetime(df3['Scheduled_Delivery_Date'])
df3.drop(df3[df3['Weight_(Kilograms)'].str.contains('See')].index, inplace = True)
df3.drop(df3[df3['Weight_(Kilograms)'].str.contains('Weight Captured Separately')].index, inplace = True)
df3['Weight_(Kilograms)'] = pd.to_numeric(df3['Weight_(Kilograms)'])
#print (df3[:5])
df3 = df3.groupby(['Scheduled_Delivery_Date','Country','ID'], as_index=False)['Weight_(Kilograms)'].mean()
df3 = df3.set_index('Scheduled_Delivery_Date') 
df3 = df3.groupby([pd.Grouper(freq="M"),'Country'])['Weight_(Kilograms)'].mean().reset_index()


# Create the Dash app
app = JupyterDash(external_stylesheets=[dbc.themes.FLATLY])

img = 'globe2.png'
test_img = base64.b64encode(open(img, 'rb').read()).decode('ascii')

#AppStructure
sidebar=html.Div(id='sidebar',children=[
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardImg(src='data:image/png;base64,{}'.format(test_img), top=True),
                dbc.CardBody([
                    html.H1("Supply Chain Health Commodity Shipment Data", style={'textAlign': 'center'}),
                    html.Br(),
                    html.H5("This data set provides supply chain health commodity shipment and pricing data. Specifically, the data set identifies Antiretroviral (ARV) and HIV lab shipments to supported countries [1].",
                           style={'textAlign': 'justify'}),
                    ])],
                color="#4f0000"
        ),        
        html.Br(),
        html.H5("Key:", style={'textAlign': 'center'}),
        html.P("Count = Number of Shipments", style={'textAlign': 'center'}),
        html.P("ACT = Artemisinin-Based Combination Therapy", style={'textAlign': 'center'}),
        html.P("ANTM = Anti-Malarial Medicine", style={'textAlign': 'center'}),
        html.P("ARV = Anti-Retroviral Treatment", style={'textAlign': 'center'}),
        html.P("HRDT = HIV Rapid Diagnostic Test", style={'textAlign': 'center'}),
        html.P("MRDT = Malarial Rapid Diagnostic Test", style={'textAlign': 'center'}),
        html.Br(),
        html.H3("[1] Data source and description: https://data.usaid.gov/HIV-AIDS/Supply-Chain-Shipment-Pricing-Data/a3rc-nmf6",style={'textAlign': 'justify'}),
        html.H3("Image source: https://thenounproject.com/browse/icons/term/delivery/")
    ])], 
        #align="center",
        #justify="center",
        style={"height": "100%"},
        className="g-0 d-flex align-items-center"
    )
])
    
    
content=html.Div(id='content', children=[   
    dbc.Row([
        html.H2("Number of Shipments by Country", style={'textAlign': 'center'}),
        dbc.Col([     
            html.Br(),
            html.H6("Select countries to display:", style={'textAlign': 'center'}),
            html.Br(),
            dcc.Dropdown(id="slct_country",
                options=sorted([{"label": dept, "value": dept} for dept in df['Country'].unique()],key = lambda x: x['label']),
                style={'color': '#111111'},
                #value=[dept for dept in df['Country'].unique()],
                value=['Afghanistan'],
                #value=[],
                multi=True),
            html.Br(),
            html.Div(id='output_container', children=[])
        ], width=3),
        dbc.Col([      
            dcc.Graph(id='map', figure={})], width=9),
    ],className="g-0"),
    
    dbc.Row([
        html.H2("Shipment Comparison by Year and Region", style={'textAlign': 'center'}),
        
        dbc.Col([
            dcc.Graph(id='sunburst', figure={}),
            html.H4(['Choose countries to compare:'],style={"text-align": "center"}),
            dcc.Dropdown(id='country_one',
                options=[{'label':x, 'value':x} for x in df3.sort_values('Country')['Country'].unique()],
                value='Nigeria',
                multi=False,
                disabled=False,
                clearable=True,
                searchable=True,
                placeholder='Choose Cuisine...',
                className='form-dropdown',
                style={
                        'height': '35px',
                        'width':'100%',
                        'display': 'inline-block',
                    },
                persistence='string',
                persistence_type='memory'),

            dcc.Dropdown(id='country_two',
                options=[{'label':x, 'value':x} for x in df3.sort_values('Country')['Country'].unique()],
                value='Vietnam',
                multi=False,
                clearable=False,
                persistence='string',
                persistence_type='session',
                style={
                        'height': '35px',
                        'width':'100%',
                        'display': 'inline-block',
                    }),

            dcc.Dropdown(id='country_three',
                options=[{'label':x, 'value':x} for x in df3.sort_values('Country')['Country'].unique()],
                value='Zimbabwe',
                multi=False,
                clearable=False,
                persistence='string',
                persistence_type='local',
                style={
                        'height': '35px',
                        'width':'100%',
                        'display': 'inline-block',
                    }),
            ],width=8),
        
        dbc.Col([
            html.H4("Level of Detail:", style={'textAlign': 'center'}),
            html.Br(),
            html.P("Global View  |  Region  |  Country  |  Product Group  |  Brand", style={'textAlign': 'center'}),
            html.P("(For example: Global View  |  Americas  |  Guyana  |  ARV  |  Generic )", style={'textAlign': 'center'}),
            dcc.Graph(id='our_graph', figure={}),
        ], width=4)
    ],className="g-0", style={"height": "60%"})
])


#BootstrapLayoutSettings
app.layout = dbc.Container(
    [
        dbc.Row(
            [
                dbc.Col(sidebar, width=2),
                dbc.Col(content, width=10)
                ]
            ),
        ],
    fluid=True
    )

@app.callback(
    [Output(component_id='output_container', component_property='children'),
    Output(component_id='map', component_property='figure'),
    Output('our_graph','figure'),
    Output(component_id='sunburst', component_property='figure')],
    [Input(component_id='slct_country', component_property='value'),
    Input('country_one','value'),
    Input('country_two','value'),
    Input('country_three','value')]
)

def update_graph(option_slctd, first_country, second_country, third_country):

    container = "Selected countries: {}".format(option_slctd)
    
    dff3=df3[(df3['Country']==first_country)|
           (df3['Country']==second_country)|
           (df3['Country']==third_country)]
    print(dff3[:5])

    dff = df.copy()

    cols = dff.columns
    cols = cols.map(lambda x: x.replace(' ', '_') if isinstance(x, (str)) else x)
    dff.columns = cols

    # make a dict with counts
    count_dict = {d:(dff['Country']==d).sum() for d in dff.Country.unique()}
    # assign that dict to a column
    dff['Shipment_count'] = [count_dict[d] for d in dff.Country]

    #new column for % of the shipment count
    dff['Perc_sc'] = (dff['Shipment_count'] / dff['Shipment_count'].sum()) * 100

    #make a dict with product groups per country
    pg_dict = dict(dff.groupby('Country')['Product_Group'].unique())
    dff['Product_groups'] = [pg_dict[d] for d in dff.Country]

    #make a dict with product sub classifications per country
    sub_dict = dict(dff.groupby('Country')['Sub_Classification'].unique())
    dff['Sub_class'] = [sub_dict[d] for d in dff.Country] 
    
    filtered_data = dff[dff['Country'].isin(option_slctd)]
    
    # Plotly Express
    fig = px.choropleth(
        data_frame=filtered_data,
        locationmode='country names',
        locations='Country',
        scope="world",
        color='Shipment_count',
        range_color=[0,1500],
        hover_data=['Country', 'Shipment_count', 'Product_groups'],
        color_continuous_scale=px.colors.sequential.Pinkyl,
        template='ggplot2',
        #height=800,
        #width=1450
    )
    
    fig.add_scattergeo(
        locations=filtered_data['Country'],
        locationmode="country names", 
        text=filtered_data['Country'],
        mode='text',
        hoverinfo='skip',
        textposition="top right",
        textfont=dict(
            size=11,
            color="#000000"
    ))
    
    fig.update_geos(
    visible=True, resolution=50, scope="world", coastlinecolor="LightGray",
    showcountries=True, countrycolor="White",
    showsubunits=True, subunitcolor="Green",
    projection_type="natural earth"
    )
    
    fig.update_layout(
        coloraxis_colorbar=dict(
        title='Shipment Count'
        ))
    
    fig.update_traces(
        marker_line_color="#ffdbda",
        hoverlabel_bgcolor="#ffd686",
        hoverlabel_font_color="black",
        hoverlabel_bordercolor="#4f0000"
    )
    
    fig2 = px.sunburst(
        dff, 
        path=['region','Country', 'Product_Group', 'Brand'], 
        template='plotly_white',
        color_discrete_map={'Asia':'#fef3b4', 'Americas':'#ffb17d', 'Africa':'#e25582'},
        color='region',
        #height=700
    )
        
    fig2.update_traces(
        maxdepth=2, 
        selector=dict(type='sunburst'),
        domain=dict(column=5),
        textfont_color='black',
        textfont_size=15
    )
    
    fig2.update_layout(
        coloraxis_colorbar=dict(
        title='Continent'
        ),
        hoverlabel=dict(
        bgcolor="#ffd686",
        font_size=15,
    ))
    
    fig3 = px.line(dff3, x="Scheduled_Delivery_Date", y="Weight_(Kilograms)", color='Country', template='seaborn',
                  color_discrete_sequence=["#19d3f3","green","#fb0d0d"])
    fig3.update_traces(mode="lines", hovertemplate=None)
    fig3.update_layout(yaxis={'title':'Weight [kg]'},
                       xaxis={'title':'Scheduled Delivery Date'},
                      title={'text':'Medicines Delivered [kg]',
                      'font':{'size':15},'x':0.5,'xanchor':'center'},
                      hovermode="x unified")
    
    return container, fig, fig2, fig3


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, host='localhost', port=8054)

/var/folders/4k/6yqgn4xx3tb4phk3x6rrp4dc0000gn/T/ipykernel_9315/3291837388.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df3['Scheduled_Delivery_Date'] = pd.to_datetime(df3['Scheduled_Delivery_Date'])


Dash app running on http://localhost:8054/
